In [1]:
import os
import subprocess
import time
import requests

# Install Python libraries
print("--- Installing Python Dependencies ---")
!pip install -q langgraph langchain langchain-ollama gradio

# Install Linux dependencies (Crucial for Ollama on Colab)
print("--- Installing System Dependencies (zstd) ---")
!sudo apt-get update && sudo apt-get install -y zstd

# Install Ollama if not present
if not os.path.exists("/usr/local/bin/ollama"):
    print("--- Installing Ollama ---")
    !curl -fsSL https://ollama.com/install.sh | sh
else:
    print("--- Ollama already installed ---")

--- Installing Python Dependencies ---
--- Installing System Dependencies (zstd) ---
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,361 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/u

In [2]:
def start_ollama_server():
    print("--- Starting Ollama Server ---")
    # Check if already running
    try:
        if requests.get("http://localhost:11434").status_code == 200:
            print("Server already running.")
            return True
    except:
        pass

    # Start server in background with EXPLICIT GPU environment variables
    # This helps prevents the 0GB GPU usage issue
    my_env = os.environ.copy()
    my_env["CUDA_VISIBLE_DEVICES"] = "0"

    subprocess.Popen(["ollama", "serve"], env=my_env, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    # Wait loop: Check if server is up every second
    print("Waiting for server to be ready...", end="", flush=True)
    for _ in range(20): # Wait up to 20 seconds
        try:
            response = requests.get("http://localhost:11434")
            if response.status_code == 200:
                print(" OK!")
                return True
        except requests.ConnectionError:
            time.sleep(1)
            print(".", end="", flush=True)

    print("\nError: Ollama server failed to start.")
    return False

# Start the server
if start_ollama_server():
    # Pull the model explicitly to ensure it exists
    print("--- Checking Model (llama3.2-vision) ---")
    !ollama pull llama3.2-vision
else:
    raise Exception("Ollama server could not start. Please restart the runtime.")


--- Starting Ollama Server ---
Waiting for server to be ready.... OK!
--- Checking Model (llama3.2-vision) ---



#                 **LANG-GRAPH:\**

In [3]:
import base64
from io import BytesIO
from PIL import Image
import gradio as gr
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages

# Initialize Model
llm = ChatOllama(model="llama3.2-vision", temperature=0)

class MedicalState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
 #Defining the Doctor's Logic by implementing Rule-based Prompting:

def doctor_node(state: MedicalState):

    # We define the prompt in a variable first for cleanliness
    prompt_content = """
      You are Dr. AI, a professional medical triage assistant whose master is Miss Riya. If the patient explicitly greets you (e.g., "Hi", "Hello"), introduce yourself as Dr. AI and mention Miss Riya. Otherwise, proceed directly to medical triage.


    ### YOUR CORE PROTOCOL ###
    1. **ANALYZE**: Read the patient's text or look at the image description.
    2. **THINK (Internal Monologue)**:
       - Check for "Red Flag" keywords (Chest pain, Numbness, Baby fever).
       - specific symptoms (Location, Severity, Duration).
       - Formulate a differential diagnosis.
    3. **RESPOND**:
       - If Emergency -> STOP and direct to Hospital/911.
       - If Vague -> Ask clarifying questions.
       - If Specific -> Provide potential causes + Home care advice + DISCLAIMER.

    ### MASTER EXAMPLES (LEARN FROM THESE PATTERNS) ###

    --- PATTERN A: THE EMERGENCY (CARDIAC) ---
    Patient: "I have a heavy pressure on my chest and my jaw hurts."
    THOUGHTS:
    - Symptoms: Retrosternal pressure ("elephant on chest"), Radiation to jaw.
    - Risk Level: CRITICAL. High probability of Myocardial Infarction (Heart Attack).
    - Action: Immediate direction to Emergency Services. Do not ask for duration.
    RESPONSE: "EMERGENCY: Chest pressure radiating to the jaw is a classic sign of a Heart Attack. Please hang up and call emergency services (911/112) IMMEDIATELY. Unlock your door and do not drive yourself to the hospital."

    --- PATTERN B: THE EMERGENCY (NEUROLOGICAL/STROKE) ---
    Patient: "My dad is slurring his words and his face looks droopy on one side."
    THOUGHTS:
    - Symptoms: Dysarthria (slurring), Facial asymmetry (droop).
    - Analysis: Matches 'FAST' protocol for Stroke (Face, Arms, Speech, Time).
    - Action: Immediate Time-Critical Emergency.
    RESPONSE: "EMERGENCY: These are signs of a STROKE. Time is brain. Call emergency services immediately. Note the exact time these symptoms started."

    --- PATTERN C: VAGUE SYMPTOM (REQUIRES TRIAGE LOOP) ---
    Patient: "My stomach hurts."
    THOUGHTS:
    - Analysis: Input is too generic.
    - Differential: Gas? Appendicitis? Ulcer? Pregnancy?
    - Missing Info: Location (Quadrant?), Type (Cramping/Burning?), Severity (1-10).
    - Action: Ask specific clarifying questions.
    RESPONSE: "I need to understand more to help you. Where exactly is the pain located (upper middle, lower right, etc.)? Is it a sharp pain or a dull ache? On a scale of 1-10, how bad is it?"

    --- PATTERN D: VISUAL DERMATOLOGY (FUNGAL) ---
    Patient: [Image Uploaded] "I have this red ring on my arm. It itches."
    THOUGHTS:
    - Visual Analysis: Circular lesion with central clearing and raised red scaly border.
    - Diagnosis: Tinea Corporis (Ringworm).
    - Differentiate: Nummular Eczema (usually no clear center).
    - Advice: Antifungal cream.
    RESPONSE: "Based on the visual pattern (ring shape with a raised border and clear center), this looks like Tinea Corporis, commonly known as Ringworm (a fungal infection). It is usually treated with over-the-counter antifungal creams like Clotrimazole. If it spreads, see a doctor."

    --- PATTERN E: VISUAL DERMATOLOGY (VIRAL/PEDIATRIC) ---
    Patient: [Image Uploaded] "My son has these fluid blisters all over and a fever."
    THOUGHTS:
    - Visual Analysis: Vesicular rash ("dewdrop on rose petal" appearance).
    - Context: Pediatric patient + Fever.
    - Diagnosis: Varicella (Chickenpox).
    - Advice: Isolation, fever management.
    RESPONSE: "The combination of fever and these specific fluid-filled blisters suggests Chickenpox (Varicella). This is highly contagious. Keep him home from school. You can use calamine lotion for itching. Avoid Aspirin. If he has trouble breathing, go to the ER."

    --- PATTERN F: ORTHOPEDICS (SPRAIN VS FRACTURE) ---
    Patient: "I rolled my ankle. It's swollen but I can walk on it a little."
    THOUGHTS:
    - Mechanism: Inversion injury.
    - Ottawa Rules: Patient is weight-bearing ("can walk a little").
    - Likelihood: Sprain > Fracture.
    - Advice: RICE Protocol.
    RESPONSE: "The fact that you can put weight on it is a good sign, suggesting a sprain rather than a severe fracture. Follow the RICE method: Rest, Ice, Compression, and Elevation. If the pain gets worse or you can't walk tomorrow, get an X-ray."

    --- PATTERN G: RED FLAG BACK PAIN (CAUDA EQUINA) ---
    Patient: "My back hurts and I can't feel my inner thighs or pee properly."
    THOUGHTS:
    - Symptoms: Lumbar pain + Saddle Anesthesia (numb thighs) + Urinary retention/incontinence.
    - Diagnosis: Cauda Equina Syndrome.
    - Risk: Permanent paralysis if untreated.
    - Action: IMMEDIATE SURGICAL EMERGENCY.
    RESPONSE: "EMERGENCY: Back pain combined with numbness in the groin/thighs or difficulty urinating is a sign of Cauda Equina Syndrome. This requires immediate surgery to prevent permanent damage. Go to the Emergency Room right now."

    --- PATTERN H: RESPIRATORY (INFECTION) ---
    Patient: "I have a green cough and a fever of 102."
    THOUGHTS:
    - Symptoms: Productive cough (purulent sputum), Pyrexia (High fever).
    - Diagnosis: Likely Bacterial Pneumonia or severe Bronchitis.
    - Action: Needs auscultation/antibiotics.
    RESPONSE: "A high fever (102°F) combined with a productive cough suggests a potential chest infection like Pneumonia. You should see a doctor for a listen to your lungs and possible antibiotics. Drink plenty of fluids."

    --- PATTERN I: PEDIATRIC FEVER RULES ---
    Patient: "My 6-week-old baby feels hot."
    THOUGHTS:
    - Patient Age: < 3 months (Neonate).
    - Symptom: Fever.
    - Protocol: Sepsis workup required. Any fever in <3mo is an emergency.
    RESPONSE: "For a baby under 3 months old, ANY fever is considered a medical emergency. Do not give Tylenol yet—take the baby to the Emergency Room or Pediatrician immediately for a check-up."

    --- PATTERN J: MENTAL HEALTH (CRISIS) ---
    Patient: "I just want to sleep and never wake up."
    THOUGHTS:
    - Analysis: Passive suicidal ideation.
    - Action: Empathetic intervention + Crisis Resources.
    RESPONSE: "I am hearing a lot of pain in your words. You are not alone. If you are feeling like you might act on these thoughts, please call or text 988 (Suicide & Crisis Lifeline) immediately. There are people who want to listen and help you through this dark moment."

    --- PATTERN K: CHRONIC/METABOLIC (DIABETES) ---
    Patient: "I am thirsty all the time and I pee every hour."
    THOUGHTS:
    - Symptoms: Polydipsia (thirst) + Polyuria (frequent urination).
    - Diagnosis: Classic signs of Diabetes Mellitus (High blood sugar).
    - Advice: Blood glucose test needed.
    RESPONSE: "Excessive thirst and frequent urination are classic signs of high blood sugar (Diabetes). You should see a primary care doctor soon to get a simple blood test (HbA1c) to check your levels."

    --- PATTERN L: OPHTHALMOLOGY (EYE) ---
    Patient: "It feels like a curtain is coming down over my right eye."
    THOUGHTS:
    - Symptom: "Curtain" vision loss, painless.
    - Diagnosis: Retinal Detachment.
    - Risk: Permanent blindness.
    - Action: Emergency Ophthalmologist referral.
    RESPONSE: "EMERGENCY: The sensation of a 'curtain' or shadow over your vision is a sign of Retinal Detachment. This is an eye emergency. Go to an ER or Eye Specialist immediately to save your vision."

    --- PATTERN M: ALLERGY (ANAPHYLAXIS) ---
    Patient: "My lips are swelling and my voice sounds hoarse after eating shrimp."
    THOUGHTS:
    - Trigger: Shellfish.
    - Symptoms: Angioedema (lip swelling) + Laryngeal edema (hoarse voice).
    - Diagnosis: Anaphylaxis.
    - Action: 911 + EpiPen.
    RESPONSE: "EMERGENCY: Swelling of the lips combined with voice changes indicates your airway may be closing (Anaphylaxis). Use an EpiPen if you have one and call emergency services immediately."

    --- END OF EXAMPLES ---

    Now, analyze the Current Patient's input below.
    Generate THOUGHTS first, then the RESPONSE.
    """

    messages_to_send = [SystemMessage(content=prompt_content)] + state['messages']

    # Add retry logic for model invocation
    try:
        response = llm.invoke(messages_to_send)
    except Exception as e:
        return {"messages": [BaseMessage(content=f"Error connecting to AI: {str(e)}", type="ai")]}

    return {"messages": [response]}

In [5]:
builder = StateGraph(MedicalState)
builder.add_node("doctor", doctor_node)
builder.set_entry_point("doctor")
builder.add_edge("doctor", END)
medical_graph = builder.compile()

In [6]:


def convert_image_to_base64(image):
    """Converts a PIL Image to a base64 string for Llama 3.2 Vision"""
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

def chat_logic(message, history, image_file):
    """
    This function bridges the Gradio UI to the LangGraph Bot.
    """
    if not message and not image_file:
        return "Please enter text or upload an image."

    # 1. Prepare the content list (Text)
    content = [{"type": "text", "text": message or "Analyze this image."}]

    # 2. If an image is uploaded, add it to the message
    if image_file is not None:
        try:
            # Gradio passes the file path string or object
            if isinstance(image_file, str):
                img = Image.open(image_file)
            else:
                img = image_file

            b64_string = convert_image_to_base64(img)
            content.append({"type": "image_url", "image_url": f"data:image/jpeg;base64,{b64_string}"})
        except Exception as e:
            return f"Error processing image: {e}"

    # 3. Create the HumanMessage
    user_msg = HumanMessage(content=content)

    # 4. Invoke LangGraph
    response = medical_graph.invoke({"messages": [user_msg]})
    bot_response = response['messages'][-1].content

    return bot_response


In [ ]:
custom_css = """
.gradio-container {
    background: linear-gradient(135deg, #f0f4f8, #e6f0fa);
    font-family: 'Inter', sans-serif;
}

h1 {
    color: #1f3c88;
    text-align: center;
    font-weight: 700;
}

.subtitle {
    text-align: center;
    color: #4a6fa5;
    font-size: 1.1rem;
    margin-bottom: 1rem;
}

.chatbot {
    border-radius: 16px !important;
    box-shadow: 0 10px 30px rgba(0,0,0,0.08);
}

footer {
    visibility: hidden;
}

.disclaimer {
    background: #fff3cd;
    border-left: 6px solid #ffcc00;
    padding: 12px;
    border-radius: 8px;
    color: #856404;
    font-size: 0.95rem;
}
"""

# -------------------------
# Gradio Blocks Layout
# -------------------------
with gr.Blocks(
    theme=gr.themes.Soft(primary_hue="blue"),
    css=custom_css,
    title="Dr. AI – Medical Diagnosis Assistant"
) as demo:

    # Header Section
    gr.Markdown("""
    # 🩺 Dr. AI
    <div class='subtitle'>Your AI-powered Medical Diagnosis Assistant</div>
    """)

    # Main Layout
    with gr.Row():
        # Chat Section
        with gr.Column(scale=3):
            # We use ChatInterface for the main chat logic
            chat_interface = gr.ChatInterface(
                fn=chat_logic,
                multimodal=True,
                textbox=gr.MultimodalTextbox(
                    placeholder="Describe symptoms or upload an image (rash, eye, report)...",
                    file_types=["image"],
                    interactive=True
                ),
                fill_height=True,
            )

        # Sidebar Info
        with gr.Column(scale=1, min_width=260):
            gr.Markdown("""
            ### 🧠 How it works
            1. Enter symptoms in text
            2. Upload an image if needed
            3. AI provides possible insights

            ### 🧪 Supported Inputs
            - Skin conditions
            - Eye issues
            - General symptoms
            - Lab report images
            """)

    # Disclaimer Section
    gr.Markdown("""
    <div class='disclaimer'>
    <strong>⚠️ Medical Disclaimer</strong><br>
    This AI system is for educational and research purposes only.
    It does <strong>NOT</strong> replace a certified medical professional.
    In emergencies, contact local medical services immediately.
    </div>
    """)

# -------------------------
# Launch App
# -------------------------
print("🚀 Launching Dr. AI UI...")
demo.launch(share=True, debug=True)

/tmp/ipython-input-954794762.py:42: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(
/tmp/ipython-input-954794762.py:42: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1052: UserWarning: Expected 3 arguments for function <function chat_logic at 0x7b38e2208360>, received 2.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/gradio/utils.py:1056: UserWarning: Expected at least 3 argume

🚀 Launching Dr. AI UI...
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2e95fede0e2f4f5aec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.12/dist-packages/gradio/helpers.py:1060: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
/usr/local/lib/python3.12/dist-packages/gradio/helpers.py:1060: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
